# **Objetivo del Cuaderno Jupyter**



Importando Librerias

In [1]:
import polars as pl
import duckdb
import plotly.express as px

In [2]:
from utils.utilidades import crear_fecha_anio_trimestre
from utils.utilidades_05_eda_internet import retorna_pies_eda_internet, retorna_barras_eda_05

Cargando la Data

In [39]:
rangos = pl.read_parquet('tablas/EDA_internet_rangos.parquet')
tecnologias = pl.read_parquet('tablas/EDA_internet_tecnologias.parquet')

***
# **Preparando la Data**

In [40]:
fechas = crear_fecha_anio_trimestre(rangos)

fechas.columns = ['anio', 'trimestre', 'Fecha']

fechas


anio,trimestre,Fecha
i64,i64,date
2023,4,2023-12-30
2020,3,2020-09-30
2023,3,2023-09-30
2022,1,2022-03-30
2018,2,2018-06-30
…,…,…
2018,1,2018-03-30
2018,4,2018-12-30
2019,1,2019-03-30


In [41]:
rangos = rangos.join(fechas, on=['anio', 'trimestre'], how='left', coalesce=True)
tecnologias = tecnologias.join(fechas, on=['anio', 'trimestre'], how='left', coalesce=True)

In [42]:
duckdb.sql("""
SELECT Fecha,
       SUM("Hasta 10 Mbps") as "Hasta 10 Mbps",
       SUM("10.01 - 30 Mbps") as "10.01 - 30 Mbps",
       SUM("30.01 - 100 Mbps") as "30.01 - 100 Mbps",
       SUM("Mayor a 100 Mbps") as "Mayor a 100 Mbps"
FROM rangos
GROUP BY Fecha;                              
""").show()

┌────────────┬───────────────┬─────────────────┬──────────────────┬──────────────────┐
│   Fecha    │ Hasta 10 Mbps │ 10.01 - 30 Mbps │ 30.01 - 100 Mbps │ Mayor a 100 Mbps │
│    date    │    int128     │     int128      │      int128      │      int128      │
├────────────┼───────────────┼─────────────────┼──────────────────┼──────────────────┤
│ 2018-03-30 │       4472395 │         2931307 │           379086 │            17681 │
│ 2022-09-30 │       1915238 │         1872595 │          3996482 │          3519740 │
│ 2019-09-30 │       4657687 │         1948115 │          2260799 │            52396 │
│ 2021-03-30 │       3745414 │         1333931 │          4134802 │           245082 │
│ 2018-12-30 │       4560306 │         3162830 │           596844 │             1287 │
│ 2023-12-30 │       1979811 │         1894152 │          3857890 │          3520216 │
│ 2022-06-30 │       2523285 │         2188513 │          5302444 │           592848 │
│ 2021-06-30 │       3693746 │         1343

In [43]:
rangos_grafico = pl.DataFrame(duckdb.sql("""
SELECT Fecha,
       SUM("Hasta 10 Mbps") as "Hasta 10 Mbps",
       SUM("10.01 - 30 Mbps") as "10.01 - 30 Mbps",
       SUM("30.01 - 100 Mbps") as "30.01 - 100 Mbps",
       SUM("Mayor a 100 Mbps") as "Mayor a 100 Mbps"
FROM rangos
GROUP BY Fecha;                              
""").to_arrow_table())

In [44]:
columns = rangos_grafico.columns[1:]
for column in columns:
    rangos_grafico = rangos_grafico.with_columns(pl.col(column).cast(pl.Int64).alias(column))
rangos_grafico.sort('Fecha')


Fecha,Hasta 10 Mbps,10.01 - 30 Mbps,30.01 - 100 Mbps,Mayor a 100 Mbps
date,i64,i64,i64,i64
2017-12-30,5015219,2366027,321924,11907
2018-03-30,4472395,2931307,379086,17681
2018-06-30,4516348,2906904,442776,586
2018-09-30,4479081,3177958,522388,1260
2018-12-30,4560306,3162830,596844,1287
…,…,…,…,…
2023-03-30,2175359,1933593,4252936,2465253
2023-06-30,1915238,1872663,3996482,3520496
2023-09-30,2055216,1917632,3730808,3425831


In [46]:
tecnologias_grafico = pl.DataFrame(duckdb.sql("""
SELECT Fecha,
       SUM(ADSL) AS ADSL,
       SUM(CABLEMODEM) AS CABLEMODEM,
       SUM(FIBRA) AS FIBRA,
       SUM(WIRELESS) AS WIRELESS,    
       SUM("Total Accesos") AS "Total Accesos", 
FROM tecnologias
GROUP BY Fecha;                                          
""").to_arrow_table())

In [47]:
columns = tecnologias_grafico.columns[1:]
for column in columns:
    tecnologias_grafico = tecnologias_grafico.with_columns(pl.col(column).cast(pl.Int64).alias(column))

tecnologias_grafico.head()

Fecha,ADSL,CABLEMODEM,FIBRA,WIRELESS,Total Accesos
date,i64,i64,i64,i64,i64
2022-09-30,1395277,6031970,2871541,557110,11091128
2021-03-30,2175211,5424782,1362976,434548,9647972
2019-09-30,3204204,4800091,557822,338241,9164684
2018-12-30,3329027,4313831,388013,264299,8473655
2020-03-30,2299457,4903674,1047817,352333,8812426


In [48]:
tecnologias_grafico.columns

['Fecha', 'ADSL', 'CABLEMODEM', 'FIBRA', 'WIRELESS', 'Total Accesos']

In [27]:
tecnologias_grafico.shape

(26, 5)

Guardo estas Tablas para tenerlas disponibles en la herramienta de Visualización sin necesidad de los cálculos previos y ser mas rápida la intaracción

In [58]:
tecnologias_grafico.write_parquet('tablas_dash/tecnologias_internet_graficos_tab2.parquet')

rangos_grafico.write_parquet('tablas_dash/rangos_internet_graficos_tab2.parquet')

***
# **Gráficos**

### **Gráfico 1:**
Tendencias de los Accesos (Según la Tecnología que los disponibiliza) del 2018 al 2024

In [55]:

df = tecnologias_grafico.sort('Fecha')
columns = ['ADSL', 'CABLEMODEM', 'FIBRA', 'WIRELESS']
names = columns
y_title = 'Cantidad de Accesos'
titulo = 'Accesos por Tecnologia'
width = 900
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']
fig = px.line()
for i in range(len(columns)):
    fig.add_scatter(x=df['Fecha'], y=df[columns[i]],
                    name=names[i], mode='lines', line=dict(color=colors[i]))
fig.update_layout(title=titulo, xaxis_title='Periodo', yaxis_title=y_title, width=width)
config = {'displayModeBar': False}
fig.show(config=config)

### **¿Que se Aprecia?**
- Hace apenas 6 Años el mercado estaba mas del `90%` se lo dividían casi a partes iguales 2 Tecnologías `Cablemodem y Adsl`.
- Los acccesos a Internet han venido en aumento, es obvio que los `casi 3 Millones de accesos que ha perdido Adsl` han sido capitalizados por `Fibra Optica` y `Cable Modem`.  

### **Gráfico 2:**
Tendencias de los Accesos (Según rango de Velocidades) del 2018 al 2024

In [54]:
df = rangos_grafico.sort('Fecha')
columns = ['Hasta 10 Mbps', '10.01 - 30 Mbps', '30.01 - 100 Mbps', 'Mayor a 100 Mbps']
names = columns
y_title = 'Cantidad de Accesos'
titulo = 'Accesos por Rango de Velocidad'
width = 900
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']
fig = px.line()
for i in range(4):
    fig.add_scatter(x=df['Fecha'], y=df[columns[i]],
                    name=names[i], mode='lines', line=dict(color=colors[i]))
fig.update_layout(title=titulo, xaxis_title='Periodo', yaxis_title=y_title, width=width)
config = {'displayModeBar': False}
fig.show(config=config)

- Según estos Datos los accesos con velocidades inferiores a los 10 MBPS representaban mas del 60% hace apenas 6 años. Pero desde esa fecha para acá ese rango de velocidades a perdido poco mas de 3 millones de acceso, una situación parecida a lo ocurrido en el Gráfico anterior con la tecnología ADSL.

In [2]:
data = pl.read_parquet('tablas_dash/data_tablero_dashboard.parquet')

### **Gráfico 3:**
Estudiando la Distribución del mercado de los accesos a Internet en el periodo Ene-Mar 2018

In [5]:
config = {'displayModeBar': False}
periodo = 'Ene - Mar 2018'
mask = data['Periodo'] == periodo
df = data.filter(mask)

fig1, fig2 = retorna_pies_eda_internet(df=df, periodo=periodo)

fig1.show(config=config)
fig2.show(config=config)


### **Gráfico 4:**
Estudiando la Distribución del mercado de los accesos a Internet en el periodo Ene-Mar 2021

In [6]:
periodo = 'Ene - Mar 2021'
mask = data['Periodo'] == periodo
df = data.filter(mask)

fig1, fig2 = retorna_pies_eda_internet(df=df, periodo=periodo)

fig1.show(config=config)
fig2.show(config=config)

### **Gráfico 5:**
Estudiando la Distribución del mercado de los accesos a Internet en el periodo Ene-Mar 2024

In [7]:
periodo = 'Ene - Mar 2024'
mask = data['Periodo'] == periodo
df = data.filter(mask)

fig1, fig2 = retorna_pies_eda_internet(df=df, periodo=periodo)

fig1.show(config=config)
fig2.show(config=config)

- Según los Datos los consumidores de Internet en el Mercado Argentino tienen clara preferencia por los servicios de alta Velocidad. Siendo en la actualidad los rangos `30 a 100 Mbps` y `Mayor a 100 Mbps` los de Mayor predominio.
- Los consumidores Argentinos no sólo muestran preferencias por los accesos de Alta velocidad, sino que están dispuestos a migrar de Servicio si esto conlleva a una mejora en la Velocidad (Basándonos en lo observado con el rango 0-10 Mbps y la tecnología Adsl) que han pasado de ser los Referentes u Lideres del Sector a ser los menos favorecidos por la escogencia del consumidor.

### KPI's Propuestos para el Estudio
- Crecimiento Trimestral de 2% en `Accesos 100 Hog` por Provincia
- Crecimiento Trimestral de 5% en `Velocidad Media de Bajada`  por Provincia
<br>

- **Nota:** 
    - El KPI `Crecimiento Trimestral de 2% de Accesos 100 Hog` fue exigido por `PROVINCIA`. Esto hace que sean 24 KPI's a mostrar.
    - La única manera que encontré de mostrar 24 KPI's en una visualización y que quede espacio para algo mas fue apoyandome en la Herramienta de los `Tooltips`
    - El KPI propuesto por mi persona fue `Crecimiento Trimestral de 5% de Velocidad Media de Bajada` Aproveché y lo mostré en los mismos tooltip que muestro los KPI's anteirores


### **Gráfico 6:**
Estudiando las Métricas de `Accesos por cada 100 Hogares` y `Velocidad Media de Bajada` en las Distintas Provincias durante el Periodo `Enero - Marzo 2018`  

In [3]:
periodo = 'Ene - Mar 2018'
mask = data['Periodo'] == periodo
df = data.filter(mask)

fig = retorna_barras_eda_05(df=df, periodo=periodo)

fig.show(config=config)


### **Gráfico 7:**
Estudiando las Métricas de `Accesos por cada 100 Hogares` y `Velocidad Media de Bajada` en las Distintas Provincias durante el Periodo `Enero - Marzo 2021` 

In [4]:
periodo = 'Ene - Mar 2021'
mask = data['Periodo'] == periodo
df = data.filter(mask)

fig = retorna_barras_eda_05(df=df, periodo=periodo)

fig.show(config=config)

### **Gráfico 8:**
Estudiando las Métricas de `Accesos por cada 100 Hogares` y `Velocidad Media de Bajada` en las Distintas Provincias durante el Periodo `Enero - Marzo 2024` 

In [5]:
periodo = 'Ene - Mar 2024'
mask = data['Periodo'] == periodo
df = data.filter(mask)

fig = retorna_barras_eda_05(df=df, periodo=periodo)

fig.show(config=config)

### **Conclusiones:**
- Los accesos de Internet no han dejado de Crecer en la última decada en Argentina.
- Existen Provincias con una penetración de servicios de Internet MUY ALTA, tal es el caso de la Ciudad Autonóma de Buenos Aires, y otras como San Juan o Chaco donde es menor
- Aunque la Media de la Velocidad de Bajada en la actualidad es alta, hay que considerar que es una `MEDIA` y por lo rápido de su crecimiento, pudiera sugerir que es todavía un porcentaje muy bajo de la población los que disponen de accesos a Internet de alta velocidad.

### **Recomendaciones:**
- Profundizar en le estudio de las localidades, ya que las poblaciones muestran Medias, donde las realidades de las Ciudades con alta densidad de Población (pudieran ser las capitales y ciudades importantes de cada provincia) tienen una realidad a otras localidades mas pequeñas y/o alejadas de GRANDES centros urbanos.  
- En sintonia con el punto anterior estudiar/utilizar la herramienta de visualización en el apartado `Mapa de Oportunidades` para ubicar localidades con baja penetracion de Internet y/o donde todavia predominen tecnologías como `Adsl` o `Wireles`, cuyos usuarios presumimos estén dispuestos a migrar a servicios de mas alta velocidad de conexión.
- Estudiar la Posibilidad de Ofrecer servicio de Internet Inalambrico de Alta Velocidad, como el Satelital, donde no se requiere una alta inversión en tendido de cablería y por ende no se estará limitado a centros con alta densidad poblacional.


